In [7]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import csv




In [8]:
csv_file = 'numerai_training_data.csv'


with open(csv_file, 'r') as f:
  save = csv.reader(f, delimiter = ',')
  raw_data = list(save)
print (raw_data[:2])

[['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6', 'feature7', 'feature8', 'feature9', 'feature10', 'feature11', 'feature12', 'feature13', 'feature14', 'feature15', 'feature16', 'feature17', 'feature18', 'feature19', 'feature20', 'feature21', 'target'], ['0.943745447280042', '0.401440492760393', '0.0524591961080846', '0.251755752856205', '0.599198006133435', '0.433078735492905', '0.154291986680751', '0.92162389945005', '0.0896378616524796', '0.117733463237376', '0.250726083709401', '0.0780808964926594', '0.823646376733102', '0.196164109166281', '0.712755076070547', '0.867629792388402', '0.409075454411293', '0.307076282705858', '0.949682259112273', '0.607615748394229', '0.41754724196422', '1']]


In [9]:
data = np.delete(raw_data, (0), axis=0)

x = np.delete(data, (-1), axis=1)
# x2 = list(map(int, x1))
print (x[:2])
y= data[:,-1]
print (y[:12])
y1 = list(map(int, y))
y1[:] = [n + 1 for n in y1]
y = np.array(y1)

print (y[:12])
print (x.shape)
print (y.shape)


[['0.943745447280042' '0.401440492760393' '0.0524591961080846'
  '0.251755752856205' '0.599198006133435' '0.433078735492905'
  '0.154291986680751' '0.92162389945005' '0.0896378616524796'
  '0.117733463237376' '0.250726083709401' '0.0780808964926594'
  '0.823646376733102' '0.196164109166281' '0.712755076070547'
  '0.867629792388402' '0.409075454411293' '0.307076282705858'
  '0.949682259112273' '0.607615748394229' '0.41754724196422']
 ['0.929175420930546' '0.863270430663643' '0.905305967406711'
  '0.698716557139708' '0.416503151341583' '0.770083788853883'
  '0.626187725493307' '0.738225125354333' '0.568634535735404'
  '0.675988513231958' '0.807240649717194' '0.700287609753836'
  '0.810102723879884' '0.784314476627852' '0.614978703187966'
  '0.598660342127264' '0.314625186603813' '0.977315200841722'
  '0.711696849396431' '0.304693361949859' '0.768249892077801']]
['1' '0' '0' '0' '0' '0' '0' '0' '1' '1' '1' '1']
[2 1 1 1 1 1 1 1 2 2 2 2]
(96320, 21)
(96320,)


In [10]:
x_train = x[:int(x.shape[0] * 0.7)]
x_validate = x[int(x.shape[0] * 0.7) + 1: int(x.shape[0] * 0.9)]
x_test = x[int(x.shape[0] * 0.9) + 1:]
print (x_train.shape)
print (x_validate.shape)

y_train = y[:int(y.shape[0] * 0.7)]
y_validate = y[int(y.shape[0] * 0.7) + 1: int(y.shape[0] * 0.9)]
y_test = y[int(y.shape[0] * 0.9) + 1:]
print (y_train.shape)
print (y_validate.shape)

(67424, 21)
(19263, 21)
(67424,)
(19263,)


In [11]:
features = 21
num_labels = 2

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, features)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(x_train, y_train)
valid_dataset, valid_labels = reformat(x_validate, y_validate)
test_dataset, test_labels = reformat(x_test, y_test)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (67424, 21) (67424, 2)
Validation set (19263, 21) (19263, 2)
Test set (9631, 21) (9631, 2)


In [25]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, features))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
  """Reusable code for making a simple neural net layer.

  It does a matrix multiply, bias add, and then uses relu to nonlinearize.
  It also sets up name scoping so that the resultant graph is easy to read,
  and adds a number of summary ops.
  """
  # Adding a name scope ensures logical grouping of the layers in the graph.
  with tf.name_scope(layer_name):
    # This Variable will hold the state of the weights for the layer
    with tf.name_scope('weights'):
      weights = weight_variable([input_dim, output_dim])
      variable_summaries(weights, layer_name + '/weights')
    with tf.name_scope('biases'):
      biases = bias_variable([output_dim])
      variable_summaries(biases, layer_name + '/biases')
    with tf.name_scope('Wx_plus_b'):
      preactivate = tf.matmul(input_tensor, weights) + biases
      tf.histogram_summary(layer_name + '/pre_activations', preactivate)
    activations = act(preactivate, 'activation')
    tf.histogram_summary(layer_name + '/activations', activations)
    return activations

  hidden1 = tf.nn.relu(nn_layer(tf_train_dataset, 21, 1024, 'layer1'))


  with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.scalar_summary('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

  logits = nn_layer(dropped, 1024, 2, 'layer2', act=tf.nn.softmax)
 
  with tf.name_scope('cross_entropy'):
    diff = tf_train_labels * tf.log(logits)
    with tf.name_scope('total'):
        cross_entropy = -tf.reduce_mean(diff)
    tf.scalar_summary('cross entropy', cross_entropy) 
    
  FLAGS.learning_rate = 0.5
   # L2 regularization for the fully connected parameters.
  #regularizers = (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) +
                  #tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
  # Add the regularization term to the loss.
  #loss += 5e-4 * regularizers   
    
  with tf.name_scope('train'):
      train_prediction = tf.train.GradientDescentOptimizer(FLAGS.learning_rate).minimize(
          cross_entropy)
    
  with tf.name_scope('accuracy'):
      with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(train_prediction, 1), tf.argmax(train_labels, 1))
      with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
      tf.scalar_summary('accuracy', accuracy)
  
    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
  merged = tf.merge_all_summaries()
  train_writer = tf.train.SummaryWriter(FLAGS.summaries_dir + '/train',
                                          session.graph)
    # test_writer = tf.train.SummaryWriter(FLAGS.summaries_dir + '/test')

  # Predictions for the training, validation, and test data.

  valid_prediction = nn_layer(tf.nn.relu(nn_layer(tf_valid_dataset, 21, 1024, 'layer1'))
                              , 1024, 2, 'layer2', act=tf.nn.softmax)
  test_prediction = nn_layer(tf.nn.relu(nn_layer(tf_test_dataset, 21, 1024, 'layer1'))
                              , 1024, 2, 'layer2', act=tf.nn.softmax)

In [16]:
def accuracy(predictions, labels):
  acc = (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])
  return acc

In [27]:
num_steps = 301

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.3 }
    summary, _, l, predictions = session.run(
      [merged, optimizer, loss, train_prediction], feed_dict=feed_dict)
    train_writer.add_summary(summary, step)
    if (step % 10 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
         valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized


NameError: name 'keep_prob' is not defined